This contains the tools for publishing a public version of first reading resolutions.

This should be used after plenary. 

It will create a folder inside Google Drive labeled 'For campus feedback' with copies of all first reading resolutions

https://developers.google.com/docs/api/quickstart/python

In [ ]:
%cd ~/Dropbox/ResolutionManagerWeb/python-scripts/

# from ResolutionManager import environment as env
from ResolutionManager.API.CredentialsManager import CredentialsManager
from ResolutionManager.Repositories.DocumentRepository import DocumentRepository
from ResolutionManager.Repositories.FileRepository import FileRepository
from ResolutionManager.Repositories.ResolutionTemplateRespository import ResolutionTemplateRepository
from ResolutionManager.Repositories.PermissionsRepository import PermissionsRepository
from ResolutionManager.Repositories.StylingRepository import StylingRepository
# import ResolutionManager.executables.make_folders_for_plenary as make_folders
# import ResolutionManager.executables.web_copy_first_readings_for_feedback as copy_first_reads
# import ResolutionManager.executables.create_resolution_from_template as create_resolution
from ResolutionManager.Models.Plenaries import Plenary
from ResolutionManager.Models.Committees import Committee
from ResolutionManager.Models.Resolutions import Resolution
import datetime
# import environment as env


from ResolutionManager.DAO.DAO import MySqlDao
from ResolutionManager.Repositories.CommitteeRepository import CommitteeRepository
from ResolutionManager.Repositories.PlenaryRepository import PlenaryRepository
from ResolutionManager.Repositories.ResolutionRepository import ResolutionRepository


In [ ]:
dao = MySqlDao()
committee_repo = CommitteeRepository(dao)
plenary_repo = PlenaryRepository(dao)
resolution_repo = ResolutionRepository(dao)
doc_repo = DocumentRepository()
file_repo = FileRepository()
style_repo = StylingRepository()

# Get document

In [ ]:
templateid = '1V8wRL5HobjyLLlISGAEvJKbVGaJcsCAwiIgNM_-oikc'
template = doc_repo.get_document(templateid)

In [ ]:
template

# Named ranges

In [ ]:
resolution = Resolution(document_id=templateid)
style_repo.get_indicies_for_named_range(resolution, 'distributionListRange')


In [ ]:
template['body']['content']

In [ ]:
template['namedRanges']

In [ ]:
titleRangeId = 'kix.mief0boal1wa'

#create named range

empty segment id = body

'elements': [{'startIndex': 20,
       'endIndex': 52,
       'textRun': {'content': 'The California State University\n',
        'textStyle': {'bold': True,
         'weightedFontFamily': {'fontFamily': 'Atkinson Hyperlegible',
          'weight': 400}}}}],

## Add range name to template

In [ ]:

def add_named_range_to_template(service, template_id, startIndex, endIndex, rangeName ):
    change = {
        "createNamedRange" :{
            "name": rangeName,
            "range": {
                "startIndex": startIndex,
                "endIndex": endIndex
            }
        }
    }
    result = service.documents().batchUpdate(
            documentId=template_id, body={'requests': change}).execute()


In [ ]:
template_id = '1V8wRL5HobjyLLlISGAEvJKbVGaJcsCAwiIgNM_-oikc'
add_named_range_to_template(doc_repo.service, template_id, 54, 62)

In [ ]:
# The Academic Senate....
group_name = {'start_index' : 1, 'end_index' : 52, 'name': 'groupName'}
add_named_range_to_template(doc_repo.service, templateid, group_name['start_index'], group_name['end_index'], group_name['name'] )

In [ ]:
dist_list = {'start_index' : 93, 'end_index' : 1290, 'name' : 'distributionListRange'}
add_named_range_to_template(doc_repo.service, templateid, dist_list['start_index'], dist_list['end_index'], dist_list['name'] )

In [ ]:
rationale = {'start_index' : 1291, 'end_index' : 1310, 'name' : 'rationaleRange'}
add_named_range_to_template(doc_repo.service, templateid, rationale['start_index'], rationale['end_index'], rationale['name'] )

## Get named range from document

In [ ]:
def get_named_ranges(document, titleRangeName=TITLE_RANGE_NAME):
    # Find the matching named ranges.
    named_range_list = document.get('namedRanges', {}).get(titleRangeName)
    if not named_range_list:
#         named_range_list = document.get('namedRanges', {})

        raise Exception('The named range is no longer present in the document.')

    # Determine all the ranges of text to be removed, and at which indices the
    # replacement text should be inserted.
    all_ranges = []
    insert_at = {}
    for named_range in named_range_list.get('namedRanges'):
        ranges = named_range.get('ranges')

        all_ranges.extend(ranges)
        # Most named ranges only contain one range of text, but it's possible
        # for it to be split into multiple ranges by user edits in the document.
        # The replacement text should only be inserted at the start of the first
        # range.
        insert_at[ranges[0].get('startIndex')] = True

    return all_ranges

In [ ]:
#no longer template
t = doc_repo.get_document('1eYxyexBr7YtdPP5704hqQgoDEa5v9a5VzPcvkIed-ZY')

In [ ]:
template['namedRanges']

In [ ]:
get_named_ranges(template)

In [ ]:
newdoc = doc_repo.get_document('1MlgjsHsGQ9k6SVOAx088sz8FjWFMzw0ZXYJMeIf7NeA')
get_named_ranges(newdoc)

In [ ]:
newdoc

In [ ]:
# footer
template.get('footers')

In [ ]:
r = resolution_repo.load_resolution(56)
r.__dict__

In [ ]:
r.end_index

In [ ]:
# replace title from template

In [ ]:
def replace_named_title_range(service, document_id, range_name, new_text):
    """Replaces the text in existing named ranges."""

    # Determine the length of the replacement text, as UTF-16 code units.
    # https://developers.google.com/docs/api/concepts/structure#start_and_end_index
    new_text_len = len(new_text.encode('utf-16-le')) / 2

    # Fetch the document to determine the current indexes of the named ranges.
    document = service.documents().get(documentId=document_id).execute()

    # Find the matching named ranges.
    named_range_list = document.get('namedRanges', {}).get(range_name)
    if not named_range_list:
        raise Exception('The named range is no longer present in the document.')

    # Determine all the ranges of text to be removed, and at which indices the
    # replacement text should be inserted.
    all_ranges = []
    insert_at = {}
    for named_range in named_range_list.get('namedRanges'):
        ranges = named_range.get('ranges')
        all_ranges.extend(ranges)
        # Most named ranges only contain one range of text, but it's possible
        # for it to be split into multiple ranges by user edits in the document.
        # The replacement text should only be inserted at the start of the first
        # range.
        insert_at[ranges[0].get('startIndex')] = True

    # Sort the list of ranges by startIndex, in descending order.
    all_ranges.sort(key=lambda r: r.get('startIndex'), reverse=True)

    # Create a sequence of requests for each range.
    requests = []
    for r in all_ranges:
        # Delete all the content in the existing range.
        requests.append({
            'deleteContentRange': {
                'range': r
            }
        })

        segment_id = r.get('segmentId')
        start = r.get('startIndex')
        if insert_at[start]:
            # Insert the replacement text.
            requests.append({
                'insertText': {
                    'location': {
                        'segmentId': segment_id,
                        'index': start
                    },
                    'text': new_text
                }
            })
            # Re-create the named range on the new text.
            requests.append({
                'createNamedRange': {
                    'name': range_name,
                    'range': {
                        'segmentId': segment_id,
                        'startIndex': start,
                        'endIndex': start + new_text_len
                    }
                }
            })

    # Make a batchUpdate request to apply the changes, ensuring the document
    # hasn't changed since we fetched it.
    body = {
        'requests': requests,
        'writeControl': {
            'requiredRevisionId': document.get('revisionId')
        }
    }
    service.documents().batchUpdate(documentId=document_id, body=body).execute()



In [ ]:
copy_id = '1eYxyexBr7YtdPP5704hqQgoDEa5v9a5VzPcvkIed-ZY'
newTitle = "Dogs are good smelling"

In [ ]:
replace_named_range(doc_repo.service, copy_id, TITLE_RANGE_NAME, newTitle)


In [ ]:
t = [1, 3, 5]
t[-1:]